In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time
import torch.nn.functional as F

/home/octav/gitrepos/tum-thesis


In [2]:
model_type = "molemb"
sys.argv = ["", "-cuda", "-model_type", model_type]
args = get_args()
outdir_suffix = "dev3"
args.n_epochs = 30 
args.init_model = "{}-test1".format(model_type)
# args.init_model = "{}-test1".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
args.batch_size = 50 

args.penalty_gumbel = False 

# for long runs
args.scale_lambdas = True
args.connectivity = True 
args.valency = True 
args.euler_characteristic_penalty = True
args.conn_lambda_start = 0.001
args.conn_lambda_end = 0.3
args.conn_lambda_epochs_start = 12
args.conn_lambda_epochs_end = 17
args.valency_lambda_start = 0.001
args.valency_lambda_end = 0.55
args.valency_lambda_epochs_start = 12
args.valency_lambda_epochs_end = 17
args.euler_lambda_start = 0.001 
args.euler_lambda_end = 0.2 
args.euler_lambda_epochs_start = 12
args.euler_lambda_epochs_end = 17
args.tau_start = 1
args.tau_end = 0.01
args.tau_epochs_start = 1
args.tau_epochs_end = 20


args.conn_penalty_function = "capped_logdet" 

print(args, '\n')

Namespace(N_transformer=6, agg_func='sum', batch_norm=False, batch_size=50, conn_lambda_end=0.3, conn_lambda_epochs_end=17, conn_lambda_epochs_start=12, conn_lambda_start=0.001, conn_penalty_function='capped_logdet', connectivity=True, connectivity_hard=False, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=True, euler_lambda_end=0.2, euler_lambda_epochs_end=17, euler_lambda_epochs_start=12, euler_lambda_start=0.001, ffn_activation='LeakyReLU', init_decoder_model='molemb-test1_decode', init_model='molemb-test1', linear_out=False, max_num_atoms=70, model_type='molemb', n_epochs=30, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=200, n_labels=1, n_layers=5, one_batch_train=False, ot_solver='emd', output_dir='mol_opt/output_dev3/molemb-test1', pc_hidden=150, penalty_gumbel=False, pred_hidden=150, rec_lambda_end=100, rec_lambda_epochs_end=50, rec_lambda_epochs_start=1, rec_l

In [4]:
model_type = "transformer-ae"
sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
args = get_args()
outdir_suffix = "dev3"
args.n_epochs = 4000 
args.init_model = "{}-test2".format(model_type)
# args.init_model = "{}-test1".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
args.batch_size = 50 

args.penalty_gumbel = False 

args.scale_lambdas = True
args.connectivity = False 
args.valency = False 
args.euler_characteristic_penalty = False
args.conn_lambda_start = 0.001
args.conn_lambda_end = 3
args.conn_lambda_epochs_start = 4001
args.conn_lambda_epochs_end = 12000
args.valency_lambda_start = 0.001
args.valency_lambda_end = 5
args.valency_lambda_epochs_start = 4001
args.valency_lambda_epochs_end = 12000
args.euler_lambda_start = 0.001 
args.euler_lambda_end = 2 
args.euler_lambda_epochs_start = 4001
args.euler_lambda_epochs_end = 12000
args.tau_start = 1
args.tau_end = 0.01
args.tau_epochs_start = 1
args.tau_epochs_end = 12000


args.conn_penalty_function = "capped_logdet" 

print(args, '\n')

Namespace(N_transformer=6, agg_func='sum', batch_norm=False, batch_size=50, conn_lambda_end=3, conn_lambda_epochs_end=12000, conn_lambda_epochs_start=4001, conn_lambda_start=0.001, conn_penalty_function='capped_logdet', connectivity=False, connectivity_hard=False, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=False, euler_lambda_end=2, euler_lambda_epochs_end=12000, euler_lambda_epochs_start=4001, euler_lambda_start=0.001, ffn_activation='LeakyReLU', init_decoder_model='transformer-ae-test2_decode', init_model='transformer-ae-test2', linear_out=False, max_num_atoms=70, model_type='transformer-ae', n_epochs=4000, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=200, n_labels=1, n_layers=5, one_batch_train=True, ot_solver='emd', output_dir='mol_opt/output_dev3/transformer-ae-test2', pc_hidden=150, penalty_gumbel=False, pred_hidden=150, rec_lambda_end=100, rec_lambda_epochs

In [5]:
# train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, same_number_atoms = True)
# train_data_loader = get_loader("molgen/data/chembl50", "train", args.batch_size, same_number_atoms = True)
# val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False)

train_data_loader = get_loader("molgen/data/chembl50", "train_split", args.batch_size, same_number_atoms = True)
val_data_loader = get_loader("molgen/data/chembl50", "val_split", args.batch_size, same_number_atoms = True)
val_data_loader = None

In [6]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = val_data_loader)

No model transformer-ae-test2 found in mol_opt/output_dev3/transformer-ae-test2! Starting from scratch.
Penalty params: tau=1.00000 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=-1 mode=[0 0 0] conn=False euler=False val=False
Epoch: 1


/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:394: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


Penalty params: tau=0.99962 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=1 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 1, train
 fgw:3.7202002
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:3.7202002
Epoch duration: 9.885762929916382
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_1
Epoch: 2
Penalty params: tau=0.99923 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=2 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 2, train
 fgw:2.4812192
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:2.4812192
Epoch duration: 9.795239448547363
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_2
Epoch: 3
Penalty params: tau=0.99885 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3, train
 fgw:4.7928128
 conn_penalty:0.00

Penalty params: tau=0.99197 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=21 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 21, train
 fgw:0.8507023
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.8507023
Epoch duration: 9.844171285629272
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_21
Epoch: 22
Penalty params: tau=0.99159 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=22 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 22, train
 fgw:0.8437998
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.8437998
Epoch duration: 9.854323625564575
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_22
Epoch: 23
Penalty params: tau=0.99121 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=23 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 23, train
 fgw:0.8472557
 conn_pe

Penalty params: tau=0.98439 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=41 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 41, train
 fgw:0.8214826
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.8214826
Epoch duration: 9.93302297592163
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_41
Epoch: 42
Penalty params: tau=0.98401 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=42 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 42, train
 fgw:0.8209784
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.8209784
Epoch duration: 9.529417037963867
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_42
Epoch: 43
Penalty params: tau=0.98363 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=43 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 43, train
 fgw:0.8184689
 conn_pen

...

Penalty params: tau=0.22376 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3901 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3901, train
 fgw:0.2705953
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2705953
Epoch duration: 9.954811811447144
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_3901
Epoch: 3902
Penalty params: tau=0.22367 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3902 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3902, train
 fgw:0.2666913
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2666913
Epoch duration: 9.81855845451355
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_3902
Epoch: 3903
Penalty params: tau=0.22359 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3903 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3903, train
 fgw

Penalty params: tau=0.22205 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3921 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3921, train
 fgw:0.2654526
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2654526
Epoch duration: 9.94911003112793
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_3921
Epoch: 3922
Penalty params: tau=0.22196 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3922 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3922, train
 fgw:0.2694370
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2694370
Epoch duration: 10.05658483505249
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_3922
Epoch: 3923
Penalty params: tau=0.22188 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3923 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3923, train
 fgw

Penalty params: tau=0.22035 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3941 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3941, train
 fgw:0.2683427
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2683427
Epoch duration: 9.983455419540405
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_3941
Epoch: 3942
Penalty params: tau=0.22026 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3942 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3942, train
 fgw:0.2707074
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2707074
Epoch duration: 9.61101222038269
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_3942
Epoch: 3943
Penalty params: tau=0.22018 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3943 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3943, train
 fgw

Penalty params: tau=0.21866 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3961 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3961, train
 fgw:0.2769365
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2769365
Epoch duration: 9.775533437728882
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_3961
Epoch: 3962
Penalty params: tau=0.21858 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3962 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3962, train
 fgw:0.2633373
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2633373
Epoch duration: 10.232763051986694
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_3962
Epoch: 3963
Penalty params: tau=0.21850 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3963 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3963, train
 f

Penalty params: tau=0.21699 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3981 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3981, train
 fgw:0.2760035
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2760035
Epoch duration: 10.260241985321045
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_3981
Epoch: 3982
Penalty params: tau=0.21691 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3982 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3982, train
 fgw:0.2680203
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2680203
Epoch duration: 10.021183013916016
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/transformer-ae-test2/model_transformer-ae-test2_3982
Epoch: 3983
Penalty params: tau=0.21683 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=3983 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3983, train
 

In [ ]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

In [6]:
for i in train_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break
    
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize(*yhat_logits)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

from mol_opt.ot_utils import FGW 
fgw_loss = FGW(alpha = 0.5)
fgw_loss(*pred_pack, tau = 1)

ArgumentError: Python argument types in
    rdkit.Chem.rdmolops.Kekulize(NoneType)
did not match C++ signature:
    Kekulize(RDKit::ROMol {lvalue} mol, bool clearAromaticFlags=False)

In [30]:
torch.optim.Adam([torch.autograd.Variable(torch.Tensor([0.]))]).param_groups[0]['lr']

0.001

In [14]:
0.9 ** (1/5)

0.9791483623609768

In [28]:
vars(args)

{'cuda': True,
 'output_dir': 'mol_opt/output_dev1/pointwise-test5',
 'tb_logs_dir': 'mol_opt/logs_dev1/pointwise-test5',
 'init_model': 'pointwise-test5',
 'init_decoder_model': 'pointwise-test5_decode',
 'task': 'qed',
 'model_type': 'pointwise',
 'one_batch_train': True,
 'batch_size': 50,
 'pred_hidden': 150,
 'pc_hidden': 100,
 'ffn_activation': 'LeakyReLU',
 'n_epochs': 1000,
 'dim_tangent_space': 40,
 'n_layers': 5,
 'n_hidden': 250,
 'n_ffn_hidden': 100,
 'linear_out': False,
 'dropout_gcn': 0.0,
 'dropout_ffn': 0.0,
 'agg_func': 'sum',
 'batch_norm': False,
 'N_transformer': 6,
 'n_ffn_transformer': 100,
 'n_heads_transformer': 10,
 'dropout_transformer': 0.1,
 'ot_solver': 'emd',
 'sinkhorn_entropy': 0.1,
 'sinkhorn_max_it': 10000,
 'connectivity': False,
 'valency': True,
 'euler_characteristic_penalty': True,
 'annealing_rate': 0.002,
 'connectivity_lambda': 0,
 'valency_lambda': 0,
 'euler_lambda': 1,
 'connectivity_hard': False,
 'valency_hard': False,
 'scale_lambdas': F

In [32]:
from tensorboardX import SummaryWriter
writer = SummaryWriter("/home/octav/gitrepos/tum-thesis/mol_opt/dev7")

In [31]:
tags = [
    "pointwise10-dev8/train_avg_euler_error/mean",
    "pointwise10-dev8/train_avg_euler_error/bot_band",
    "pointwise10-dev8/train_avg_euler_error/top_band"
]
# writer.add_custom_scalars_marginchart(tags)


In [2]:
# writer.add_scalar("plm2/plm2", 0, 1)
# writer.add_scalar("plm2/plm2", 1, 2)
# writer.add_scalar("plm2/plm2", 0, 3)
# writer.add_scalar("plm2/plm2", 1, 4)
tags = ["plm", "plm", "plm"]
# layout = {"plm" : {"plm" : ["Margin", tags]}}
layout = {"plm1" : {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]},
          "plm2": {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]}}
writer.add_custom_scalars(layout)

In [33]:
writer._get_file_writer().add_summary()

TypeError: add_summary() missing 1 required positional argument: 'summary'

In [34]:
writer.logdir

'/home/octav/gitrepos/tum-thesis/mol_opt/dev6'

In [11]:
writer.close()

In [4]:
import tensorboardX
tensorboardX.__version__

'2.1'

In [6]:
from tensorboardX.summary import custom_scalars

In [16]:
writer._get_file_writer().add_summary(custom_scalars(layout), global_step = 3)
writer.flush()

In [34]:
writer.add_scalars('plm', {'plm0' : 0.0, 'plmt' : 0.1, 'plmb' : -0.1}, 3)
writer.flush()

In [ ]:
tau = 1
symbols_nll, charges_nll, bonds_nll = F.gumbel_softmax(tau = tau, dim=1, logits = symbols_logits), F.gumbel_softmax(tau=tau,dim=1,logits=charges_logits), F.gumbel_softmax(tau=tau,dim=1, logits = bonds_logits)

In [ ]:
np.sort(symbols_nll.mean(axis = 0).detach().numpy())

In [ ]:
list(molopt.opt0.parameters())[0]

In [5]:
molopt.state_dict()

NameError: name 'molopt' is not defined

In [ ]:
model_dict = torch.load('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8')

In [ ]:
model_dict["model"]['opt0.weight']

In [ ]:
molopt2 = MolOpt(args)

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt2.load_state_dict(model_dict["model"])

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt3,args3 = load_model('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8', MolOpt, args.device)

In [ ]:
list(molopt3.opt0.parameters())[0]

In [ ]:
fgw_loss(*pred_pack, tau = 1)